# Tutorial: Learn machine learning models

This tutorial demonstrates how to learn machine learning models to value on-the-ball actions of football players with the open-source [VAEP framework](https://github.com/ML-KULeuven/socceraction) using the publicly available [Wyscout match event dataset](https://figshare.com/collections/Soccer_match_event_dataset/4415000). The Wyscout dataset includes data for the 2017/2018 English Premier League, the 2017/2018 Spanish Primera División, the 2017/2018 German 1. Bundesliga, the 2017/2018 Italian Serie A, the 2017/2018 French Ligue 1, the 2018 FIFA World Cup, and the UEFA Euro 2016. Covering 1,941 matches, 3,251,294 events and 4,299 players, the dataset is large enough to train machine-learning models and obtain robust ratings for the players.

This tutorial demonstrates the following four steps:
1. Split the dataset into a training set and a test set.
2. Construct the baseline classifiers by using conservative hyperparameters for the learning algorithm.
3. Optimize the classifiers by tuning the hyperparameters for the learning algorithm.
4. Construct the final classifiers using the optimal hyperparameters for the learning algorithm.

**Conventions:**
* Variables that refer a `DataFrame` object are prefixed with `df_`.
* Variables that refer a collection of `DataFrame` objects (e.g., a list, a set or a dict) are prefixed with `dfs_`.

**References:**
* Tom Decroos, Lotte Bransen, Jan Van Haaren, and Jesse Davis. "[Actions Speak Louder than Goals: Valuing Player Actions in Soccer.](https://arxiv.org/abs/1802.07127)" In *Proceedings of the 25th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining*, pp. 1851-1861. 2019.
* Luca Pappalardo, Paolo Cintia, Alessio Rossi, Emanuele Massucco, Paolo Ferragina, Dino Pedreschi, and Fosca Giannotti. "[A Public Data Set of Spatio-Temporal Match Events in Soccer Competitions.](https://www.nature.com/articles/s41597-019-0247-7)" *Scientific Data 6*, no. 1 (2019): 1-15.

**Optional:** If you run this notebook on Google Colab, then uncomment the code in the following cell and execute the cell.

In [ ]:
# !pip install tables==3.6.1
# !pip install socceraction
# !pip install scikit-plot

**Optional:** If you run this notebook on Google Colab and wish to store all data in a Google Drive folder, then uncomment the code in the following cell and execute the cell.

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# %mkdir -p '/content/gdrive/My Drive/Friends of Tracking/'
# %cd '/content/gdrive/My Drive/Friends of Tracking/'

In [ ]:
from tqdm.notebook import tqdm

%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd  # version 1.0.3

from sklearn.metrics import brier_score_loss, roc_auc_score  # version 0.22.2
from sklearn.model_selection import train_test_split, GridSearchCV  # version 0.22.2
from sklearn.calibration import CalibratedClassifierCV  # version 0.22.2

from scikitplot.metrics import plot_calibration_curve

from xgboost import XGBClassifier  # version 1.0.2

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

# Load dataset

This third tutorial assumes that the `spadl.h5` HDF5 file as well as the `features.h5` and `labels.h5` files have been created for a set of games in the first or second tutorial.

This third tutorial only uses features that have been generated in the first tutorial. However, you are strongly encouraged to toy around with the additional features from the second tutorial and to try out your own features to improve the accuracy of the predictive machine learning models!

## Load games

In [ ]:
df_games = pd.read_hdf('spadl.h5', key='games')

**Optional:** If you plan to produce results for a particular season in the Wyscout match event dataset (e.g., 2017/2018 English Premier League), you should consider leaving that season out of the dataset to avoid the same game states from appearing in both the training set and the test set. If you would like to only include a particular subset of games, then uncomment the code in the following cell, adapt the selector and execute the cell. The example selector will select all games that were played in the 2017/2018 English Premier League.

In [ ]:
# df_games = df_games[
#     df_games['competition_id'] == 364
# ]

In [ ]:
df_games.tail(10)

## Load features

Load the *features* for the selected games and combine them into the `df_features` `DataFrame` object.

In [ ]:
dfs_features = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_features = pd.read_hdf('features.h5', key=f'game_{game_id}')
    df_features['game_id'] = game_id
    dfs_features.append(df_features)
df_features = pd.concat(dfs_features).reset_index(drop=True)

In [ ]:
df_features.tail(10)

## Load labels

Load the *labels* for the selected games and combine them into the `df_labels` `DataFrame` object.

In [ ]:
dfs_labels = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']  
    df_labels = pd.read_hdf('labels.h5', key=f'game_{game_id}')
    df_labels['game_id'] = game_id
    dfs_labels.append(df_labels)
df_labels = pd.concat(dfs_labels).reset_index(drop=True)

In [ ]:
df_labels.tail(10)

# Split dataset

To accurately predict the labels for game states, the challenge is to train a robust machine learning model that generalizes well from *observed* game states (e.g., game states in games that have been played already) to *unobserved* game states (e.g., game states in games that are yet to be played). Hence, the machine learning model needs to capture interactions between features of game states that are not too specific to identify *similar* game states yet specific enough to distinguish between *dissimilar* game states.

Typically, a machine learning model is said to be *overfitted* on the data if the interactions are too specific (i.e., apply to just a few game states) and is said to be *underfitted* on the data if the interactions are too general (i.e., apply to about all game states). To avoid underfitting, we define an expressive set of features and use a machine learning algorithm that learns a model that can capture complex relationships between these features. To avoid overfitting, we apply [cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html) to assess how well a candidate model would perform in practice and apply regularization techniques to keep the learned model as simple as possible.

In this tutorial, we will apply the following methodology:
1. Split the available data into a training set and a test set. The training set will be used to learn the machine learning model, whereas the test set will be kept aside until the very end to assess the real-world performance of the learned model.
2. Learn models with different values for the hyperparameters of the learning algorithm on the training set by adopting a $k$-fold cross-validation setup:
  * Split the training set in $k$ random folds or subsets of equal size.
  * Use $k$-1 training folds to train the model and compute the evaluation metric on the remaining validation fold.
  * Repeat $k$ times until each fold has served as the validation fold once.
3. Select the values for the hyperparameters that yield the best performance across the $k$ validation folds according to the evaluation metric.
4. Train the final model on the full training set using the optimal values for the hyperparameters.
5. Apply the final model on the test set and compute the evaluation metric.

The following cell splits the available data into a training set and test set using the `train_test_split` function.

* **Test set size:** The `test_size` parameter controls the number of examples in the test set. The challenge is to find an appropriate balance between the number of training examples and the number of test examples. Typically, more training examples yield better models, whereas more test examples yield more reliable evaluation metrics.
* **Random state:** The `random_state` parameter sets the *seed* for the random number generator. By setting a *seed*, the train-test split will be the same for each execution of the `train_test_split` function, which is important for reproducing the results and comparing different models.
* **Stratification:** The `stratify` parameter enforces a *stratified* train-test split according to the provided class label. By doing so, the distribution of the provided class label will be the same in the training set and the test set, which can be helpful to obtain a well-calibrated model. Since we will be using two different class labels (i.e., `scores` and `concedes`), we use a concatenation of both class labels for the stratification procedure.

In [ ]:
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(
    df_features,
    df_labels,
    test_size=0.10,
    random_state=42,
    stratify=df_labels['scores'].astype(str) + '_' + df_labels['concedes'].astype(str)
)

The following cells inspect whether the training set and test set have the same proportion of positive and negative examples for each class.

In [ ]:
df_y_train['scores'].mean()

In [ ]:
df_y_test['scores'].mean()

In [ ]:
df_y_train['concedes'].mean()

In [ ]:
df_y_test['concedes'].mean()

**Note:** In a real-world scenario where more data is available, you should consider respecting the chronological order of the games to construct the training set, validation set and test set. For instance, use the data for the 2016/2017 and 2017/2018 seasons to train the models, use the data for the 2018/2019 season to tune the models, and use the data for the 2019/2020 season to obtain the results.

# Construct baseline classifiers

The following cell provides a list of features that the machine learning algorithm will consider to train the model.

In [ ]:
features = [
    'start_dist_to_goal_a0',
    'end_dist_to_goal_a0',
    'start_dist_to_goal_a1',
    'end_dist_to_goal_a1',
    'start_dist_to_goal_a2',
    'end_dist_to_goal_a2',
    'start_angle_to_goal_a0',
    'end_angle_to_goal_a0',
    'start_angle_to_goal_a1',
    'end_angle_to_goal_a1',
    'start_angle_to_goal_a2',
    'end_angle_to_goal_a2',
    'team_1',
    'team_2'
]

The following cell provides a list of class labels for which the machine learning algorithm will train a model.

**Note:** The `concedes` class label has been commented to speed up the execution of the entire notebook.

In [ ]:
labels = [
    'scores',
#     'concedes'
]

## Train classifiers

The following cell trains an XGBoost classifier for each label using *conservative* hyperparamters for the learning algorithm, which will serve as *baseline* models.

In [ ]:
%%time
models = {}
for label in tqdm(labels):
    model = XGBClassifier(
        n_estimators=10,
        max_depth=3
    )
    model.fit(
        X=df_X_train[features],
        y=df_y_train[label]
    )
    models[label] = model

## Estimate probabilities

The following cell estimates the probabilities for each label using the trained *baseline* models.

In [ ]:
dfs_predictions = {}
for label in tqdm(labels):
    model = models[label]
    probabilities = model.predict_proba(
        df_X_test[features]
    )
    predictions = probabilities[:, 1]
    dfs_predictions[label] = pd.Series(predictions, index=df_X_test.index)
df_predictions = pd.concat(dfs_predictions, axis=1)

In [ ]:
df_predictions.head(10)

## Evaluate probabilities

### Compute base rate probabilities

The following cell computes the *base rate* or *prior probability* of each class label in the training set. We use the *base rate* as a naive estimate for each example in the test set being true to establish a baseline for the evaluation metrics.

In [ ]:
df_base_rates = pd.DataFrame({
    label:np.full(len(df_y_test[label]), df_y_train[label].mean()) for label in labels
})

In [ ]:
df_base_rates.head(10)

### Compute Brier score loss for goal scored model

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the base rate predictions.

In [ ]:
brier_score_loss(
    y_true=df_y_test['scores'],
    y_prob=df_base_rates['scores']
)

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the predictions by the learned model.

In [ ]:
brier_score_loss(
    y_true=df_y_test['scores'],
    y_prob=df_predictions['scores']
)

### Compute Brier score loss for goal conceded model

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the base rate predictions.

In [ ]:
# brier_score_loss(
#     y_true=df_y_test['concedes'],
#     y_prob=df_base_rates['concedes']
# )

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the predictions by the learned model.

In [ ]:
# brier_score_loss(
#     y_true=df_y_test['concedes'],
#     y_prob=df_predictions['concedes']
# )

### Plot calibration curve and probability histogram

The following cell creates a plot to show both a calibration curve and a probability histogram.

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(8, 8),
    gridspec_kw={
        'height_ratios': [3, 1]
    }
)

The following cell plots the *calibration curve* for the trained model.

In [ ]:
plot_calibration_curve(
    y_true=df_y_test['scores'],
    probas_list=[df_predictions['scores'].tolist()],
    clf_names=['Goal scored model'],
    n_bins=10,
    ax=ax1
)

The following cell plots the histogram of the predictions produced by the trained model. Clearly, the majority of the examples is *negative*, which means that the start of the calibration curve is extremely important.

In [ ]:
df_predictions['scores'].plot.hist(
    range=(0, 1),
    bins=10,
    ax=ax2
)

# Optimize classifiers

In order to train *accurate* models, we perform a *grid search* over different combinations of parameter values for the most important hyperparameters for the learning algorithm. We will focus on the number of estimators and the maximum depth of the decision trees although more hyperparameters influence the performance of the trained models. The more combinations of parameter values need to be explored, the longer the *grid search* will take.

Furthermore, restricting the number of estimators (i.e., the number of decision trees) and the maximum depth of the decision trees is an important mechanism to reduce the complexity of the trained models and thus also to avoid overfitting on the training data. The more decision trees that the learning algorithm can use and the deeper these decision trees can become, the more opportunities the learning algorithm has to *memorize* the training data rather than to discover patterns that generalize to the unseen test data.

## Train classifiers

The following cell trains an XGBoost classifier for each label by trying different combinations of hyperparameters.

In [ ]:
%%time
models_cv = {}
for label in tqdm(labels):
    model = GridSearchCV(
        estimator=XGBClassifier(),
        param_grid={
            'n_estimators': [50, 100],
            'max_depth': [3, 4]
        },
        scoring='neg_brier_score',
        refit=True,  # train final model on full training set using best hyperparameters
        verbose=10,
        n_jobs=1
    )
    model.fit(
        X=df_X_train[features],
        y=df_y_train[label]
    )
    models_cv[label] = model

**Note:** We have considered a manually selected set of features to represent the game states. In addition to optimizing the hyperparameters for the learning algorithm, we could also optimize the set of features to be considered by the learning algorithm. However, the XGBoost algorithm should be able to figure out by itself which features are most important to include in the model by the nature of the algorithm.

## Estimate probabilities

The following cell estimates the probabilities for each label using the trained *baseline* models.

In [ ]:
dfs_predictions_cv = {}
for label in tqdm(labels):
    model = models_cv[label]
    probabilities = model.predict_proba(
        df_X_test[features]
    )
    predictions = probabilities[:, 1]
    dfs_predictions_cv[label] = pd.Series(predictions, index=df_X_test.index)
df_predictions_cv = pd.concat(dfs_predictions, axis=1)

## Evaluate probabilities

### Compute Brier score loss for goal scored model

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the base rate predictions.

In [ ]:
brier_score_loss(
    y_true=df_y_test['scores'],
    y_prob=df_base_rates['scores']
)

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the predictions by the learned model.

In [ ]:
brier_score_loss(
    y_true=df_y_test['scores'],
    y_prob=df_predictions_cv['scores']
)

### Compute Brier score loss for goal conceded model

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the base rate predictions.

In [ ]:
# brier_score_loss(
#     y_true=df_y_test['concedes'],
#     y_prob=df_base_rates['concedes']
# )

The following cell computes the [Brier loss score](https://en.wikipedia.org/wiki/Brier_score) for the predictions by the learned model.

In [ ]:
# brier_score_loss(
#     y_true=df_y_test['concedes'],
#     y_prob=df_predictions_cv['concedes']
# )

### Plot calibration curve and probability histogram

The following cell creates a plot to show both a calibration curve and a probability histogram.

In [ ]:
fig_cv, (ax1_cv, ax2_cv) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(8, 8),
    gridspec_kw={
        'height_ratios': [3, 1]
    }
)

The following cell plots the *calibration curve* for the trained model.

In [ ]:
plot_calibration_curve(
    y_true=df_y_test['scores'],
    probas_list=[df_predictions_cv['scores'].tolist()],
    clf_names=['Goal scored model'],
    n_bins=10,
    ax=ax1_cv
)

The following cell plots the histogram of the predictions produced by the trained model. Clearly, the majority of the examples is *negative*, which means that the start of the calibration curve is extremely important.

In [ ]:
df_predictions_cv['scores'].plot.hist(
    range=(0, 1),
    bins=10,
    ax=ax2_cv
)

## Optional: Calibrate probabilities

If the trained model produces poorly calibrated probability estimates, the probability estimates can be re-calibrated using [CalibratedClassifierCV](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html), which performs probability calibration with isotonic regression or logistic regression.

# Construct final classifiers

Once we have found the *best* feature set and *best* hyperparameters for the learning algorithm, we can learn the final model.

1. If we use `GridSearchCV` and the `refit` parameter was set to `True`, we can retrieve the *final* model, which has been re-trained on the entire training set, by accessing the `best_estimator_` attribute of the object.
2. We can manually train the *final* model by creating a `XGBClassifier` object using the *best* hyperparameters and calling the `fit` method with the entire training set.

## Option 1: Retrieve classifier from grid search

In [ ]:
model_scores = models_cv['scores']

The following cell shows the *best* hyperparameter combination that was found using the *grid search*.

In [ ]:
pd.Series(
    model_scores.best_params_
)

The following cell shows the full results of the *grid search*.

In [ ]:
pd.DataFrame(
    model_scores.cv_results_
)

The following cell shows the hyperparameters that were used to re-train the model on the entire training set.

In [ ]:
pd.Series(
    model_scores.best_estimator_.get_params()
)

The following cell retrieves the final `XGBClassifier` object from the `GridSearchCV` object.

In [ ]:
model_scores_final = model_scores.best_estimator_

In [ ]:
model_scores_final

## Option 2: Train classifier using optimal hyperparameters

The following cell constructs a new `XGBoostClassifier` object using the *best* hyperparameters that were found by the *grid search*.

In [ ]:
model_scores_final = XGBClassifier(
    n_estimators=100,
    max_depth=4
)

The following cell fits the `XGBoostClassifier` object on the entire training set.

In [ ]:
model_scores_final.fit(
    X=df_X_train[features],
    y=df_y_train[label]
)